<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


In [2]:
import os
from bertopic import BERTopic
import json
RESULTS_HOME = '/kaggle/input/instance-json/results/results'
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

2024-07-04 07:02:04.564398: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 07:02:04.564497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 07:02:04.683402: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


topics.json:   0%|          | 0.00/10.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

topic_embeddings.safetensors:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

ctfidf_config.json:   0%|          | 0.00/4.66M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
'''topic, prob = topic_model.transform("This is an incredible movie!")
topic_model.topic_labels_[topic[0]]'''
from tqdm import tqdm

In [5]:
def process_batch(posts_batch):
    t, _ = topic_model.transform(posts_batch)
    return [topic_model.topic_labels_[i] for i in t]

In [21]:
aligned_count = 0
total_count = 1
#for instance_name in os.listdir(RESULTS_HOME):
instance = json.load(open(os.path.join(RESULTS_HOME, 'mastodon.social.json'), 'r'))
posts = [post['text'] for post in tqdm(instance['records'], desc='Pull post' ) if post['language'] == 'en']
aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
tags = []
for tagl in tqdm(aux, desc='Processing tags'):
    for tag in tagl:
        tags.append(tag['name'])
        
batch_size = 1000
batches = [posts[i:i + batch_size] for i in range(0, len(posts), batch_size)]

topic_path = os.path.join('/kaggle/working', 'fullgas_topics.txt')

all_topics = []
for batch in tqdm(batches, desc='Processing batches'):
    topics = process_batch(batch)
    all_topics.extend(topics)
    
    with open(topic_path, 'a') as f:
        for topic in tqdm(topics, desc='Save Topic'):
            f.write(f'{topic}\n')
    
if len(tags) > 0:
    total_count += len(tags)
    for tag in tqdm(tags, desc='Checking tag alignment'):
        for topic in all_topics:
            if tag in topic:
                aligned_count += 1
                break
                
output_path = os.path.join('/kaggle/working', 'count_results.txt')
with open(output_path, 'w') as file:
    file.write(f'Aligned Count: {aligned_count}\n')
    file.write(f'Total Count: {total_count}\n')
    file.write(f'Alignment / Total : {aligned_count / total_count:.4f}\n')
                
print(aligned_count)
print(total_count)
print(aligned_count/total_count)

Processing batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-07-04 07:35:51,598 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

Processing batches:  10%|█         | 1/10 [00:00<00:06,  1.34it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-07-04 07:35:52,378 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

Processing batches:  20%|██        | 2/10 [00:01<00:06,  1.30it/s]

...

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-07-04 07:35:58,671 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

Processing batches:  90%|█████████ | 9/10 [00:07<00:00,  1.10it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-07-04 07:35:59,625 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.

quem-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 645267.61it/s]

nowplaying-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 414514.26it/s]

newmusic-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 593220.19it/s]

newmusic-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 1146924.80it/s]

nowplaying-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 1284508.01it/s]

newmusic-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 891324.14it/s]

nowplaying-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 942476.69it/s]

newmusic-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 892481.06it/s]

nowplaying-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 1079309.33it/s]

exs6em-alignment Topic: 100%|██████████| 10000/10000 [00:00<00:00, 1261824.31it/s]

4070e9-alignment 

KeyboardInterrupt: 

In [15]:
instance = json.load(open(os.path.join(RESULTS_HOME, 'mastodon.social.json'), 'r'))
print(len(tags))
print(len(tags2))

400000
414704


In [6]:
from tqdm import tqdm
c = 0
for instance_name in tqdm(os.listdir(RESULTS_HOME)):
    instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
    posts = [post['text'] for post in instance['records'] if post['language'] == 'en']
    aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
    tags = []
    for tagl in aux:
        for tag in tagl:
            tags.append(tag['name'])
    if len(tags)>0:
        c+= len(posts)
    
print(c)
#Numero di post totali da analizzare 18387780

100%|██████████| 788/788 [12:02<00:00,  1.09it/s]

18387780


In [20]:
#!rm -r /kaggle/working

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


rm: cannot remove '/kaggle/working': Device or resource busy
